In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
class LoanDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [3]:
class LoanPreprocessor:
    def __init__(self):
        # Categorical features for one-hot encoding
        self.categorical_features = [
            'person_home_ownership', 
            'loan_intent', 
            'loan_grade', 
            'cb_person_default_on_file'
        ]
        
        # Numerical features for scaling
        self.numerical_features = [
            'person_age', 
            'person_income', 
            'person_emp_length', 
            'loan_amnt', 
            'loan_int_rate', 
            'loan_percent_income', 
            'cb_person_cred_hist_length'
        ]
        
        # Preprocessors
        self.preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), self.numerical_features),
                ('cat', OneHotEncoder(handle_unknown='ignore'), self.categorical_features)
            ])
    
    def fit_transform(self, X, y=None):
        # Fit and transform the data
        X_processed = self.preprocessor.fit_transform(X)
        return X_processed
    
    def transform(self, X):
        # Transform new data
        X_processed = self.preprocessor.transform(X)
        return X_processed

In [4]:
class LoanNeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(LoanNeuralNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.network(x)

In [5]:
def train_model(model, train_loader, criterion, optimizer, device, epochs=50):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(features)
            loss = criterion(outputs.squeeze(), labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        if epoch % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}')